In [ ]:
%config Completer.use_jedi = False

In [2]:
import dash
from jupyter_dash import JupyterDash

from dash import html
from dash import dcc

from logging import PlaceHolder
from dash.dependencies import Input, Output, State

import plotly.express as px
import pandas as pd

import os
import warnings

In [3]:
app = JupyterDash(external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css'])

# Load and Clean the Dataset

## Players Data

In [4]:
#Reading Cleaned players Dataset 
cleaned_merged_seasons = pd.read_csv('./Data/cleaned_merged_seasons.csv',low_memory=False)

In [5]:
cleaned_merged_seasons.head(3)

,Unnamed: 0,season_x,name,position,team_x,assists,bonus,bps,clean_sheets,creativity,...,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW
0,0,2016-17,Aaron Cresswell,DEF,NaN,0,0,0,0,0.0,...,2.0,0.0,0,0,0,0,55,False,0,1
1,1,2016-17,Abdoulaye Doucouré,MID,NaN,0,0,0,0,0.0,...,1.0,0.0,0,0,0,0,50,False,0,1
2,2,2016-17,Adam Forshaw,MID,NaN,0,0,3,0,1.3,...,1.0,0.0,1,0,0,0,45,True,1,1


In [6]:
cleaned_merged_seasons["was_home"].replace({True: "Home", False: "Away"}, inplace=True)

In [7]:
season=cleaned_merged_seasons.season_x.unique()
players=cleaned_merged_seasons.name.unique()

## Teams Data

In [8]:
needed_columns = ['Season', 'Team', 'h_a', 'scored','missed',
                  'pts', 'wins', 'draws', 'loses', 'result']

In [9]:
df_2019_2020 = []
for dirname, _, filenames in os.walk('./Data/2019-20/'):
    for filename in filenames:
        df_2019_2020.append(pd.read_csv(os.path.join(dirname, filename)))

In [10]:
df_2020_2021 = []
for dirname, _, filenames in os.walk('./Data/2020-21/'):
    for filename in filenames:
        df_2020_2021.append(pd.read_csv(os.path.join(dirname, filename)))

In [11]:
df_2021_2022 = []
for dirname, _, filenames in os.walk('./Data/2021-22/'):
    for filename in filenames:
        df_2021_2022.append(pd.read_csv(os.path.join(dirname, filename)))

In [12]:
df_2019_2020 = pd.concat(df_2019_2020, ignore_index=True)

In [13]:
df_2020_2021 = pd.concat(df_2020_2021, ignore_index=True)

In [14]:
df_2021_2022 = pd.concat(df_2021_2022, ignore_index=True)

In [15]:
df_2019_2020['Season'] = '2019-2020'
df_2020_2021['Season'] = '2020-2021'
df_2021_2022['Season'] = '2021-2022'

In [16]:
all_season = pd.concat([df_2019_2020, df_2020_2021, df_2021_2022], ignore_index=True)

In [17]:
all_season = all_season[needed_columns]

In [18]:
warnings.filterwarnings("ignore")

**Convert h_a and result columns**

In [19]:
h_a_dic = {'h': 'Home', 'a': 'Away'}
result_dic = {'w': 'Win', 'l': 'Loss', 'd': 'Draw'}

In [20]:
all_season['h_a'] = all_season['h_a'].map(h_a_dic)
all_season['result'] = all_season['result'].map(result_dic)

In [21]:
all_season.columns = ['Season', 'Team', 'Home_Away', 'scored','Conceded',
                      'pts', 'wins', 'draws', 'loses', 'result']

**Numerical and Categorical Columns**

In [22]:
# Select numerical columns
num_cols = [cname for cname in all_season.columns 
            if all_season[cname].dtype in ['int64', 'float64']]

cat_cols = list(set(all_season.columns) - set(num_cols) - {'result'})

print("Numerical Columns:", num_cols, '', sep='\n')
print("Categorical Columns:", cat_cols, '', sep='\n')

Numerical Columns:
['scored', 'Conceded', 'pts', 'wins', 'draws', 'loses']

Categorical Columns:
['Season', 'Team', 'Home_Away']



In [23]:
all_season.head()

,Season,Team,Home_Away,scored,Conceded,pts,wins,draws,loses,result
0,2019-2020,Arsenal,Away,1,0,3,1,0,0,Win
1,2019-2020,Arsenal,Home,2,1,3,1,0,0,Win
2,2019-2020,Arsenal,Away,1,3,0,0,0,1,Loss
3,2019-2020,Arsenal,Home,2,2,1,0,1,0,Draw
4,2019-2020,Arsenal,Away,2,2,1,0,1,0,Draw


In [24]:
Seasons = list(all_season['Season'].unique())
Teams = list(all_season['Team'].unique())

# Dash App Layout

In [25]:
colors = {
    'background': '#37003c',
    'sec_background':'#02884e',
    'text': '#fff'
}

In [26]:
app.layout = html.Div(children=[
    #Header Items
    html.H1('Fantasy Premier League Visualization',
            style={'font_family': 'Helvetica Neue','color': colors['text'],'backgroundColor': colors['background']
                   ,'textAlign':'center'}),
    html.Img(src=app.get_asset_url('wide_logo.png'), style={'height':'25%', 'width':'25%'}),
    html.Br(), # New line
    
    #Players performace items
    html.H2('Players Performance',
            style={'width': '30%','height': '20%','border':'1px solid', 'border-radius': 10
                   ,'color': colors['text'],'backgroundColor': colors['sec_background']
                   ,'textAlign':'center','display': 'inline-block'}),
    html.Div([
        html.Div([
            html.Pre(children="Select Player", style={"fontSize":"150%"}),
            dcc.Dropdown(
                    id="dropdown",
                    options=[{"label": x, "value": x} for x in players],
                    value='Mohamed Salah',
                    clearable=False,
                    )
                ], className='six columns'),

        html.Div([
            html.Pre(children="Select Season", style={"fontSize": "150%"}),
            dcc.Dropdown(
                    id="dropdown2",
                    options=[{"label": x, "value": x} for x in season],
                    value=season[-1],
                    clearable=False,
                    )
                ], className='six columns'),
    ], className='row'),
    dcc.Graph(id="bar-chart", figure={}),
    html.Br(), # New line
    
    #Teams performace items
    html.H2('Teams Performance',style={'width': '30%','height': '20%','border':'1px solid', 'border-radius': 10,
                                       'color': colors['text'],'backgroundColor': colors['sec_background']
                                       ,'textAlign':'center','display': 'inline-block'}),
    html.Div([
        html.Div([
            html.Pre(children="Select Team", style={"fontSize":"150%"}),
            dcc.Dropdown(id = 'TeamsDropdown',
                    options = [{'label': str(team), 'value': str(team)} for team in Teams], value="Arsenal")
                ], className='six columns'),

        html.Div([
            html.Pre(children="Select Season", style={"fontSize": "150%"}),
            dcc.Dropdown(id = 'SeasonsDropdown',
                    options = [{'label': str(season), 'value': str(season)} for season in Seasons], value="2019-2020")
                ], className='six columns'),
    ], className='row'),
    # Div's for Graphs and Text
    html.Div(children=[
        html.Div([
            html.Pre('Team Result during One Season', style={'textAlign':'center',"fontSize":"150%"}),
            dcc.Graph(id = 'resultGraph', figure = {}),
        ], className='four columns'),
        
        html.Div([
            html.Pre('Team Goals Scored/Conceded during One Season', style={'textAlign':'center',"fontSize":"150%"}),
            dcc.Graph(id = 'goalsGraph', figure = {}),
        ], className='four columns'),
        
        html.Div([
            html.P(id = 'TeamPoints', style={'textAlign':'center'}),
            html.P(id = 'GoalScored', style={'textAlign':'center'}),
            html.P(id = 'GoalConceded', style={'textAlign':'center'})
        ], className='four columns',style = { 'marginTop': 225, 'align-items': 'center',
                                             'justify-content': 'center','display': 'inline-block', 'vertical-align': 'middle'})
    ], className='row'),
    html.H4('© 2021 Mahmoud Eidarous & Mostafa Toema',
            style={'width': '40%','height': '20%','border':'1px solid', 'border-radius': 10
                   ,'color': colors['text'],'backgroundColor': colors['background']
                   ,'textAlign':'center','display': 'inline-block'})
    
],style={'textAlign':'center'})

@app.callback(
    Output("bar-chart", "figure"), 
    Output('resultGraph','figure'),
    Output('goalsGraph', 'figure'),
    Output('TeamPoints','children'),
    Output('GoalScored', 'children'),
    Output('GoalConceded','children'),
    [Input("dropdown", "value"),
     Input('dropdown2','value'),
     Input('TeamsDropdown','value'),
     Input('SeasonsDropdown','value')])
def update_bar_chart(player,season_e, team, season):
    mask = cleaned_merged_seasons[(cleaned_merged_seasons["name"] == player) & (cleaned_merged_seasons["season_x"] == season_e)]
    
    fig = px.bar(mask, x="GW", y="total_points", color="was_home",text='opp_team_name',
                 hover_data=['minutes','selected'],labels={'total_points':'Total Points','GW':'Game Week Number','was_home': 'Home or Away'},
                 title="Player Fantasy Points Per Game Week")
    fig.update_layout(
    title={'text': 'Player Fantasy Points Per Game Week','y':0.9,'x':0.5,
            'xanchor': 'center','yanchor': 'top'})
    
    result_df = all_season[(all_season['Team'] == team) & (all_season['Season'] == season)]\
                           .groupby(['result', 'Home_Away'])[['result']].count().reset_index(level= [1])
    
    goals_df = all_season[(all_season['Team'] == team) & (all_season['Season'] == season)]\
                           .groupby(['Home_Away'])[['scored','Conceded']].sum()
    
    result_fig = px.bar(result_df, x = result_df.index, y = 'result', barmode = 'group', color='Home_Away')
    
    goals_fig = px.bar(goals_df, x =  goals_df.index, y = ['scored', 'Conceded'], barmode = 'group')
    
    totalPoints = all_season[(all_season['Team'] == team) & (all_season['Season'] == season)]['pts'].sum()
    
    TeamPoints = 'Total Points of {} in Season {} is {}'.format(team, season, totalPoints)
    GoalScored = 'Goal Scored of {} in Season {} is {}'.format(team, season, goals_df['scored'].sum())
    GoalConceded = 'Goal Conceded of {} in Season {} is {}'.format(team, season, goals_df['Conceded'].sum())
    
    return fig,result_fig, goals_fig, TeamPoints, GoalScored, GoalConceded



In [27]:
app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/
